# Libraries



In [ ]:

import tensorflow as tf
import os
from matplotlib import pyplot as plt
from os import listdir

from os.path import isfile, join
import numpy as np
import keras
import scipy.io
from scipy.io import loadmat
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from keras import backend as K
from os import listdir
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Layer
from tensorflow.keras.constraints import MinMaxNorm,NonNeg
import scipy 
from scipy import interpolate
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
%cd /content/drive/My Drive/DeepFusion
from Read_Data_set import *
from Filter_pattern import *
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

# Patterned Layer

In [ ]:
def get_color_bases(wls):
    SG = [0.0875,0.1098,0.1157,0.1245,0.1379,0.1561,0.1840,0.2458,0.3101,0.3384,0.3917\
        ,0.5000,0.5732,0.6547,0.6627,0.624,0.5719,0.5157,0.4310,0.3470,0.2670,0.1760\
        ,0.1170,0.0874,0.0754,0.0674,0.0667,0.0694,0.0567,0.0360,0.0213]   # green color

    SB = [0.2340,0.2885,0.4613,0.5091,0.5558,0.5740,0.6120,0.6066,0.5759,0.4997\
        ,0.4000,0.3000,0.2070,0.1360,0.0921,0.0637,0.0360,0.0205,0.0130,0.0110\
        ,0.0080,0.0060,0.0062,0.0084,0.0101,0.0121,0.0180,0.0215,0.0164,0.0085\
        ,0.0050]                                                           # blue color

    SR = [0.1020,0.1020,0.0790,0.0590,0.0460,0.0360,0.0297,0.0293,0.0310,0.03230\
          ,0.0317,0.0367,0.0483,0.0667,0.0580,0.0346,0.0263,0.0487,0.1716,0.4342\
          ,0.5736,0.5839,0.5679,0.5438,0.5318,0.5010,0.4810,0.4249,0.2979,0.1362\
          ,0.0651]                                                         # red color

    SC = [0.1895,0.2118,0.1947,0.1835,0.1839,0.1921,0.2137,0.2751,0.3411,0.3707\
        ,0.4234,0.5367,0.6215,0.7214,0.7207,0.6586,0.5982,0.5644,0.6026,0.7812\
        ,0.8406,0.7599,0.6849,0.6312,0.6072,0.5684,0.5477,0.4943,0.3546,0.1722\
        ,0.0864]                                                          # cyan color

    x_wvls = np.linspace(399e-9,701e-9,len(SG))

    fg = interpolate.interp1d(x_wvls, SG)
    fr = interpolate.interp1d(x_wvls, SR)
    fc = interpolate.interp1d(x_wvls, SC)
    fb = interpolate.interp1d(x_wvls, SB)
                                                                                                                               
    fr = fr(wls)
    fg = fg(wls)
    fc = fc(wls)
    fb = fb(wls)

    fr = tf.convert_to_tensor(fr, dtype=tf.float32)
    fg = tf.convert_to_tensor(fg, dtype=tf.float32)
    fc = tf.convert_to_tensor(fc, dtype=tf.float32)
    fb = tf.convert_to_tensor(fb, dtype=tf.float32)

    fr = tf.expand_dims(tf.expand_dims(fr, 0), 0)
    fg = tf.expand_dims(tf.expand_dims(fg, 0), 0)
    fc = tf.expand_dims(tf.expand_dims(fc, 0), 0)
    fb = tf.expand_dims(tf.expand_dims(fb, 0), 0)


    return fr,fg,fc,fb

def multispectral_Im(x):
  ds = x[3]
  dm = x[2]
  filters = x[1]
  x = x[0]

  print('Original Filters Shape: ', filters.shape)

  filters = tf.keras.backend.repeat_elements(filters, rep=ceil(1/dm), axis=1)
  filters = tf.keras.backend.repeat_elements(filters, rep=ceil(1/dm), axis=0)
  filters = tf.image.resize(filters,[x.shape[1],x.shape[2]])
  print('Replicated Filter Shape: ', filters.shape)
  filters = tf.broadcast_to(filters, shape=tf.shape(x))
  output = tf.expand_dims(tf.math.reduce_sum(tf.multiply(x, filters), axis=-1),-1)

  return output




    
class MultispectralAcq(Layer):
    def __init__(self, output_dim=(256, 256, 12), input_dim=(256, 256, 12), w_ini=400, w_final=700, ds = 1, dm = 1/9, sd = 0.5, train = True,**kwargs):
        self.output_dim = output_dim
        self.M = input_dim[0]
        self.N = input_dim[1]
        self.L = input_dim[2]
        self.Lp = ceil(input_dim[2]*sd)
        self.ds = ds
        self.dm = dm
        self.sd = sd
        self.train = train

        fr,fg,fc,fb=get_color_bases(np.linspace(w_ini, w_final, ceil(input_dim[2]*sd))*1e-9)

        self.fr = tf.expand_dims(fr, 0)
        self.fg = tf.expand_dims(fg, 0)
        self.fc = tf.expand_dims(fc, 0)
        self.fb = tf.expand_dims(fb, 0)


        super(MultispectralAcq, self).__init__(**kwargs)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'output_dim': self.output_dim,
            'input_dim': self.input_dim,
            'fr': self.fr,
            'fg': self.fg,
            'fc': self.fc,
            'fb': self.fb,
            'ds': self.ds,
            'dm': self.dm,
            'train': slef.train})
      
        return config

    def build(self, input_shape):
        sd = self.sd
        train =self.train
        M = self.M
        N = self.N
        Lp = self.Lp
        dm = self.dm
        Mm = int(M*dm)
        Nm = int(N*dm)
        wr = (np.random.rand(Mm, Nm, 1))
        wg = (np.random.rand(Mm, Nm, 1))
        wb = (np.random.rand(Mm, Nm, 1))
        wc = (np.random.rand(Mm, Nm, 1))
        wt = wr + wg + wb + wc
        wg = tf.constant_initializer(np.divide(wg, wt))
        wb = tf.constant_initializer(np.divide(wb, wt))
        wc = tf.constant_initializer(np.divide(wc, wt))
        wr = tf.constant_initializer(np.divide(wr, wt))  
        print(train)
        if train: 
          print('Trainable MCFA Layer')
         
          self.wr = self.add_weight(name='wr', shape=(1, Mm, Nm, 1),
                                    initializer=wr, trainable=True, constraint=NonNeg())
          self.wg = self.add_weight(name='wg', shape=(1, Mm, Nm, 1),
                                    initializer=wg, trainable=True, constraint=NonNeg())
          self.wb = self.add_weight(name='wb', shape=(1, Mm, Nm, 1),
                                    initializer=wb, trainable=True, constraint=NonNeg())
          self.wc = self.add_weight(name='wc', shape=(1, Mm, Nm, 1),
                                    initializer=wc, trainable=True, constraint=NonNeg())
        else:
          print('No Trainable MCFA Layer') 

          self.wr = self.add_weight(name='wr', shape=(1, Mm, Nm, 1),
                                    initializer=wr, trainable=False)
          self.wg = self.add_weight(name='wg', shape=(1, Mm, Nm, 1),
                                    initializer=wg, trainable=False)
          self.wb = self.add_weight(name='wb', shape=(1, Mm, Nm, 1),
                                    initializer=wb, trainable=False)
          self.wc = self.add_weight(name='wc', shape=(1, Mm, Nm, 1),
                                    initializer=wc, trainable=False)


    def call(self, inputs, **kwargs):
        
        

        M = self.M
        N = self.N
        L = self.L
        Lp = self.Lp

        inputs = tf.expand_dims(tf.reshape(tf.image.resize(tf.expand_dims(tf.reshape(inputs,[M*N,L]),-1),[M*N,Lp]),[M,N,Lp]),0)
        
        ds = self.ds
        dm = self.dm
        Mm = int(M*dm)
        Nm = int(N*dm)
        if self.trainable:
          
          wt = self.wr + self.wg + self.wb + self.wc
          wr = tf.math.divide(self.wr, wt)
          wg = tf.math.divide(self.wg, wt)
          wb = tf.math.divide(self.wb, wt)
          wc = tf.math.divide(self.wc, wt)
        else:
          
          wr = self.wr
          wg = self.wg
          wb = self.wb
          wc = self.wc



        H = tf.multiply(wr, self.fr) + tf.multiply(wg, self.fg) + tf.multiply(wb, self.fb) + tf.multiply(wc, self.fc)
        

        if ds < dm:
            y = tf.multiply(H,inputs)
            y = tf.image.resize(y,[int(M*ds), int(N*ds)])
            y = tf.expand_dims(tf.reduce_sum(y,axis=-1),axis=-1)
        elif  ds > dm:

            H = tf.keras.backend.repeat_elements(H, rep=int(ceil(1/dm)), axis=2)
            H = tf.keras.backend.repeat_elements(H, rep=int(ceil(1/dm)), axis=1)
           
            H = tf.image.resize(H,[M,N])
           
            y = tf.multiply(H,inputs) 
            y = tf.reduce_sum(y,axis=-1)
            y = tf.expand_dims(y,axis=-1)
            


        elif ds == dm:
            y = tf.multiply(H,inputs)
            y = tf.expand_dims(tf.reduce_sum(y,axis=-1),axis=-1)

        #y = tf.expand_dims(y,axis=-1)
        X = None
        for i in range(Lp):
          if X is not None:

            X = tf.concat([X,y],-1)
          else:
            X = y

        X = tf.transpose(X,[0,1,2,3])
        
        
        
        X = tf.multiply(H, X)


        X = X / tf.math.reduce_max(X) 
            
        return X,H,y

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)



# CASSI Layer

In [ ]:
class Reg_Binary_0_1(tf.keras.regularizers.Regularizer):
    def __init__(self, parameter=10):
        self.parameter = tf.keras.backend.variable(parameter,name='parameter')
    def __call__(self, x):
        regularization = self.parameter*(tf.reduce_sum(tf.multiply(tf.square(x),tf.square(1-x))))
        return regularization

    def get_config(self):
        return {'parameter': float(tf.keras.backend.get_value(self.parameter))}

class CASSI_layer(Layer):
    def __init__(self, output_dim=(256, 256, 31), input_dim=(512, 512, 31), compression=1, parm1=0.5, parm2=0.5,
                 type_code='Binary_0', trans=0.5, type_reg='Physical', kern=256, noise=40, batch_size=1, HO = (0.25, 0.5, 0.25),shots = 1,decimation = 0.5,train = True, **kwargs):
        self.output_dim = output_dim
        self.input_dim = input_dim
        #self.shots = int(np.round((compression*(input_dim[0]*input_dim[1]*input_dim[2]))/(input_dim[0]*(input_dim[1]+input_dim[2]-1))))
        self.shots = shots
        #tf.print(self.shots)

        self.parms1 = parm1
        self.parms2 = parm2
        self.type_code = type_code
        self.type_reg = type_reg
        self.trans = trans
        self.kern = kern
        self.noise = noise
        self.batch_size = batch_size
        self.HO = tf.expand_dims(tf.expand_dims(tf.expand_dims(tf.expand_dims(HO,0),0),-1),-1)
        self.decimation = decimation
        self.train = train

        if self.type_code == 'Binary_0':
            if self.type_reg =='Physical':
                self.my_regularizer = Reg_Binary_0_1(parm1)


        super(CASSI_layer, self).__init__(**kwargs)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'output_dim': self.output_dim,
            'input_dim': self.input_dim,
            'shots': self.shots,
            'parms1': self.parms1,
            'parms2': self.parms2,
            'type_code': self.type_code,
            'trans': self.trans,
            'kern': self.kern,
            'noise': self.noise,
            'batch_size': self.batch_size,
            'my_regularizer': self.my_regularizer,
            'type_reg': self.type_reg,
            'HO': self.HO,
            'train': self.train})
        return config

    def build(self, input_shape):

        if self.train:
          print('Trainable CASSI Layer')
          if self.type_code == 'Binary_0':
            
              H_init = np.random.rand(1, self.kern, self.kern, self.shots)
              #H_init = np.random.normal(0, 1, (1, self.kern, self.kern, 1, self.shots)) / np.sqrt(
                  #self.kern * self.kern)+0.5
              H_init = tf.constant_initializer(H_init)
              self.H = self.add_weight(name='H', shape=(1,self.kern, self.kern, 1,self.shots), initializer=H_init, trainable=True,regularizer=self.my_regularizer)
        else:
          print('No Trainable CASSI Layer')
          H_init = tf.constant_initializer(np.random.randint(0,2,size=(1, self.kern, self.kern, self.shots)))
          self.H = self.add_weight(name='H', shape=(1,self.kern, self.kern, 1,self.shots), initializer=H_init,trainable=False)
  


    def call(self, inputs, **kwargs):
        decimation = self.decimation
        K = self.shots
        H = self.H
        L = self.input_dim[2]
        M = int(decimation*self.input_dim[0])
        HO = self.HO
        # High Order CASSI Sensing Model
        inputs = tf.image.resize(inputs,[M,M])
        inputs = tf.expand_dims(inputs,-1)
        Image2 = tf.reduce_sum(tf.nn.conv3d(inputs,HO,strides=[1,1,1,1,1],padding='SAME'),axis=-1)

        Aux1 = tf.multiply(H,tf.expand_dims(Image2,-1))
        Aux1 = tf.pad(Aux1, [[0, 0], [0, 0], [0, L - 1], [0, 0],[0, 0]])

        Y = None
        for i in range(L):
            Tempo = tf.roll(Aux1, shift=i, axis=2)
            if Y is not None:
                Y = tf.concat([Y, tf.expand_dims(Tempo[:, :, :, i], -1)], axis=4)
            else:
                Y = tf.expand_dims(Tempo[:, :, :, i], -1)
        Y = tf.reduce_sum(Y, -1)



        # CASSI Transpose model (x = H'*y)

        X = None
        for i in range(L):
            Tempo = tf.roll(Y, shift=-i, axis=2)
            if X is not None:
                X = tf.concat([X, tf.expand_dims(Tempo[:, :, 0:M], -1)], axis=4)
            else:
                X = tf.expand_dims(Tempo[:, :, 0:M], -1)


        # High Order
        X = tf.transpose(X,[0,1,2,4,3])

        X2 = None
        for i in range(K):
          Aux2 = tf.expand_dims(X[:,:,:,:,i],-1)
          if X2 is not None:
            X2 =tf.concat([X2,tf.expand_dims(tf.reduce_sum(tf.nn.conv3d_transpose(Aux2,HO,[1,M,M,L,1],strides=[1,1,1,1,1],padding='SAME'),axis=-1),-1)],4)
          else:
            X2 = tf.expand_dims(tf.reduce_sum(tf.nn.conv3d_transpose(Aux2,HO,[1,M,M,L,1],strides=[1,1,1,1,1],padding='SAME'),axis=-1),-1)


        X = tf.multiply(H, X)
        X = tf.reduce_sum(X,4)

        X = X / tf.math.reduce_max(X)


        return X,H,Y

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)




# Unrolling Functions

In [ ]:


def TransposeCASSI(yh,H,M,L,shots,decimation):
  M = int(round(M.numpy()*decimation.numpy()))
  HO = [0.25,5,0.25]
  HO = tf.expand_dims(tf.expand_dims(tf.expand_dims(tf.expand_dims(HO,0),0),-1),-1)
  X = None
  for i in range(L):
      Tempo = tf.roll(yh, shift=-i, axis=2)
      if X is not None:
          X = tf.concat([X, tf.expand_dims(Tempo[:, :, 0:M], -1)], axis=4)
      else:
          X = tf.expand_dims(Tempo[:, :, 0:M], -1)


  # High Order
  X = tf.transpose(X,[0,1,2,4,3])

  X2 = None
  for i in range(shots):
    Aux2 = tf.expand_dims(X[:,:,:,:,i],-1)
    if X2 is not None:
      X2 =tf.concat([X2,tf.expand_dims(tf.reduce_sum(tf.nn.conv3d_transpose(Aux2,HO,[1,M,M,L,1],strides=[1,1,1,1,1],padding='SAME'),axis=-1),-1)],4)
    else:
      X2 = tf.expand_dims(tf.reduce_sum(tf.nn.conv3d_transpose(Aux2,HO,[1,M,M,L,1],strides=[1,1,1,1,1],padding='SAME'),axis=-1),-1)


  X = tf.multiply(H, X)
  X = tf.reduce_sum(X,4)

  return X / tf.math.reduce_max(X)


def ForwardCASSI(inputs,H,decimation):
  [B,M,N,L] = inputs.shape

  M = round(M*decimation.numpy())

  M = int(M)
  


  HO = [0.25, 0.5, 0.25]
  HO = tf.expand_dims(tf.expand_dims(tf.expand_dims(tf.expand_dims(HO,0),0),-1),-1)
  # High Order CASSI Sensing Model
  inputs = tf.image.resize(inputs,[M,M])
  inputs = tf.expand_dims(inputs,-1)
  Image2 = tf.reduce_sum(tf.nn.conv3d(inputs,HO,strides=[1,1,1,1,1],padding='SAME'),axis=-1)

  Aux1 = tf.multiply(tf.broadcast_to(H,shape = tf.shape(inputs)),tf.expand_dims(Image2,-1))
  Aux1 = tf.pad(Aux1, [[0, 0], [0, 0], [0, L - 1], [0, 0],[0, 0]])

  Y = None
  for i in range(L):
      Tempo = tf.roll(Aux1, shift=i, axis=2)
      if Y is not None:
          Y = tf.concat([Y, tf.expand_dims(Tempo[:, :, :, i], -1)], axis=4)
      else:
          Y = tf.expand_dims(Tempo[:, :, :, i], -1)
  return tf.reduce_sum(Y, -1)


def ForwardMultispectral(inputs,H,sd,dm,ds):
  
  [B,M,N,L] = inputs.shape
  
  Lp = int(round(L*sd.numpy()))
  inputs = tf.expand_dims(tf.reshape(tf.image.resize(tf.expand_dims(tf.reshape(inputs,[M*N,L]),-1),[M*N,Lp]),[M,N,Lp]),0)
  y = tf.expand_dims(tf.reduce_sum(tf.multiply(tf.broadcast_to(H,shape= tf.shape(inputs)),inputs),-1),-1)

  
  return y
  
def TransposeMultispectral(y,H,Lp):

  X = None
  for i in range(Lp):
    if X is not None:

      X = tf.concat([X,y],-1)
    else:
      X = y

  X = tf.transpose(X,[0,1,2,3])
  


  X = tf.multiply(H, X)


  return X / tf.math.reduce_max(X)

class Mu_parameter(keras.layers.Layer):
    def __init__(self, units=1, input_dim=32):
        super(Mu_parameter, self).__init__()
        w_init = tf.keras.initializers.Constant(value=0)
        self.w = tf.Variable(
            initial_value=w_init(shape=(units,1), dtype="float32"),
            trainable=True, constraint = NonNeg()
        )

    def call(self, inputs):
        return tf.multiply(self.w, inputs) 

class Lambda_parameter(keras.layers.Layer):
    def __init__(self, units=1, input_dim=32):
        super(Lambda_parameter, self).__init__()
        w_init = tf.keras.initializers.Constant(value=0)
        self.w = tf.Variable(
            initial_value=w_init(shape=(units,1), dtype="float32"),
            trainable=True, constraint = NonNeg()
        )

    def call(self, inputs):
        return tf.multiply(self.w, inputs) 


def GradientMultispectral(x):
  X = x[0]
  ym = x[1]
  Hm = x[2]
  sd = x[3]
  dm = x[4]
  ds = x[5]
  Lp = x[6]

  
  ym_e = ForwardMultispectral(X,Hm,sd,dm,ds)
  rm = ym_e-ym
  Gm = TransposeMultispectral(rm,Hm,Lp)
  return Gm
def GradientCASSI(x):
  X = x[0]
  yh = x[1]
  M = x[2]
  L = x[3]
  shots = x[4]
  decimation = x[5]
  Hh = x[6]
  yh_e = ForwardCASSI(X,Hh,decimation)
  rh = yh_e-yh
  Gh = TransposeCASSI(rh,Hh,M,L,shots,decimation)
  return Gh


# Unrolling Network


In [ ]:

def FusionNetwork(pretrained_weights=None, input_size=(512, 512, 31), depth=31, dm = 1, ds = 1, w_ini=400, w_final=700, batch_size=4,decimation = 0.5,sd=0.5,shots=1,trainable=True,parameter = 0.5):
  inputs = Input(shape=input_size)

  # Customer layers 

  [Xm,Hm,ym] = MultispectralAcq(output_dim=(input_size[0],input_size[1],1),input_dim=input_size, w_ini=w_ini, w_final=w_final, ds = ds, dm = dm,sd = 0.5,train=trainable)(inputs) 

  [Xh,Hh,yh] = CASSI_layer(output_dim=(512, 512, 31), input_dim=(512, 512, 31), compression=1, parm1=parameter, parm2=0.5,
                 type_code='Binary_0', trans=0.5, type_reg='Physical', kern=256, batch_size=1, HO = (0.25, 0.5, 0.25),shots = 1,decimation=0.5, train =trainable)(inputs)
  
  #Initialization
  #Xh = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(Xh)
  Xh = UpSampling2D([int(1/decimation),int(1/decimation)])(Xh)

  Xm = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(Xm) 
  X0 = Add(name='X0')([Xh,Xm])

  #Stage 1
  stage_1_m = Lambda(GradientMultispectral)([X0,ym,Hm,sd,dm,ds,16]) 
  stage_1_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_1_m) 

  stage_1_h = Lambda(GradientCASSI)([X0,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  stage_1_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_1_h)

  Up1 =  UpSampling2D([2,2])(X0)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)
  
  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X0,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)
  
  add_x = Subtract()([X0,conv_r5])

  PP_1 = Mu_parameter()(add_x)

  Add_G1 = Add()([stage_1_h,stage_1_m,PP_1])

  GS_1 = Lambda_parameter()(Add_G1)

  X1 = Subtract(name='X1')([X0,GS_1])
  
  #Stage 2
  stage_2_m = Lambda(GradientMultispectral)([X1,ym,Hm,sd,dm,ds,16]) 
  stage_2_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_2_m) 

  stage_2_h = Lambda(GradientCASSI)([X1,yh,input_size[0],input_size[2],shots,decimation,Hh])
  #stage_2_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_2_h) 
  stage_2_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_2_h)

  Up1 =  UpSampling2D([2,2])(X1)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X1,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X1,conv_r5])
  

  PP_2 = Mu_parameter()(add_x)

  Add_G2 = Add()([stage_2_h,stage_2_m,PP_2])

  GS_2 = Lambda_parameter()(Add_G2)

  X2 = Subtract(name='X2')([X1,GS_2])

  #Stage 3

  stage_3_m = Lambda(GradientMultispectral)([X2,ym,Hm,sd,dm,ds,16]) 
  stage_3_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_3_m) 

  stage_3_h = Lambda(GradientCASSI)([X2,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_3_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_3_h) 
  stage_3_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_3_h)

  Up1 =  UpSampling2D([2,2])(X2)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)
  
  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X2,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X2,conv_r5])

  PP_3 = Mu_parameter()(add_x)

  Add_G3 = Add()([stage_3_h,stage_3_m,PP_3])

  GS_3 = Lambda_parameter()(Add_G3)

  X3 = Subtract(name='X3')([X2,GS_3])

  # Stage 4
  stage_4_m = Lambda(GradientMultispectral)([X3,ym,Hm,sd,dm,ds,16]) 
  stage_4_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_4_m) 

  stage_4_h = Lambda(GradientCASSI)([X3,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_4_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_4_h) 
  stage_4_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_4_h)

  Up1 =  UpSampling2D([2,2])(X3)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)
 
  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X3,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X3,conv_r5]) 

  PP_4 = Mu_parameter()(add_x)

  Add_G4 = Add()([stage_4_h,stage_4_m,PP_4])

  GS_4 = Lambda_parameter()(Add_G4)

  X4 = Subtract(name='X4')([X3,GS_4])

  # Stage 5
  stage_5_m = Lambda(GradientMultispectral)([X4,ym,Hm,sd,dm,ds,16]) 
  stage_5_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_5_m) 

  stage_5_h = Lambda(GradientCASSI)([X4,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_5_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_5_h) 
  stage_5_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_5_h)

  Up1 =  UpSampling2D([2,2])(X4)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X4,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X4,conv_r5])

  PP_5 = Mu_parameter()(add_x)

  Add_G5 = Add()([stage_5_h,stage_5_m,PP_5])

  GS_5 = Lambda_parameter()(Add_G5)

  X5 = Subtract(name='X5')([X4,GS_5])
  
  # Stage 6
  stage_6_m = Lambda(GradientMultispectral)([X5,ym,Hm,sd,dm,ds,16]) 
  stage_6_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_6_m) 

  stage_6_h = Lambda(GradientCASSI)([X5,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_6_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_6_h) 
  stage_6_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_6_h)

  Up1 =  UpSampling2D([2,2])(X5)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X5,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)
  
  add_x = Subtract()([X5,conv_r5]) 

  PP_6 = Mu_parameter()(add_x)

  Add_G6 = Add()([stage_6_h,stage_6_m,PP_6])

  GS_6 = Lambda_parameter()(Add_G6)

  X6 = Subtract(name='X6')([X5,GS_6])
  

  # Stage 7


  stage_7_m = Lambda(GradientMultispectral)([X6,ym,Hm,sd,dm,ds,16]) 
  stage_7_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_7_m) 

  stage_7_h = Lambda(GradientCASSI)([X6,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_7_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_7_h) 
  stage_7_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_7_h)

  Up1 =  UpSampling2D([2,2])(X6)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
    
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X6,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X6,conv_r5])

  PP_7 = Mu_parameter()(add_x)

  Add_G7 = Add()([stage_7_h,stage_7_m,PP_7])

  GS_7 = Lambda_parameter()(Add_G7)

  X7 = Subtract(name='X7')([X6,GS_7])
  
  #Stage 8


  stage_8_m = Lambda(GradientMultispectral)([X7,ym,Hm,sd,dm,ds,16]) 
  stage_8_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_8_m) 

  stage_8_h = Lambda(GradientCASSI)([X7,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_7_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_7_h) 
  stage_8_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_8_h)

  Up1 =  UpSampling2D([2,2])(X7)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = Add()([X7,conv_r4])

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X7,conv_r5])

  PP_8 = Mu_parameter()(add_x)

  Add_G8 = Add()([stage_8_h,stage_8_m,PP_8])

  GS_8 = Lambda_parameter()(Add_G8)

  X8 = Subtract(name='X8')([X7,GS_8])

  #Stage 9

  stage_9_m = Lambda(GradientMultispectral)([X8,ym,Hm,sd,dm,ds,16]) 
  stage_9_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_9_m) 

  stage_9_h = Lambda(GradientCASSI)([X8,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_9_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_7_h) 
  stage_9_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_9_h)

  Up1 =  UpSampling2D([2,2])(X8)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X8,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X8,conv_r5])

  PP_9 = Mu_parameter()(add_x)

  Add_G9 = Add()([stage_9_h,stage_9_m,PP_9])

  GS_9 = Lambda_parameter()(Add_G9)

  X9 = Subtract(name='X9')([X8,GS_9])
 
  #Stage 10

  stage_10_m = Lambda(GradientMultispectral)([X9,ym,Hm,sd,dm,ds,16]) 
  stage_10_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_10_m) 

  stage_10_h = Lambda(GradientCASSI)([X9,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_10_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_10_h) 
  stage_10_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_10_h)

  Up1 =  UpSampling2D([2,2])(X9)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)
  
  conv_r4 = Add()([X9,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X9,conv_r5])

  PP_10 = Mu_parameter()(add_x)

  Add_G10 = Add()([stage_10_h,stage_10_m,PP_10])

  GS_10 = Lambda_parameter()(Add_G10)

  X10 = Subtract(name='X10')([X9,GS_10])
  
  


  
  model = Model(inputs,X10)


  if (pretrained_weights):
      model.load_weights(pretrained_weights)
      print('loading weights generator')

  return model

# Custom Callbacks, Losses and Metrics

In [ ]:

def custom_loss():
  def lossimage_1(y_true, y_pred):
    val = tf.reduce_mean(tf.norm(y_pred - y_true, ord='fro', axis=(1, 2))) + 10*tf.reduce_mean(tf.norm(y_pred - y_true, ord=2, axis=-1))
    return val
  return lossimage_1



def PSNR_Metric(y_true, y_pred):
  return tf.reduce_mean(tf.image.psnr(y_true,y_pred,1))

def ERGAS_Metric(y_true, y_pred):
  return tf.reduce_mean(tf.keras.losses().update_state(y_true,y_pred))

def SSIM_Metric(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_pred,y_true,1))

def SNR_metric(y_true, y_pred):
  return tf.reduce_sum(10*tf.math.log())



def custom_loss_3():
  def lossimage_3(y_true, y_pred):
    val = tf.divide(tf.reduce_sum((y_true-y_pred)**2)**0.5,tf.reduce_sum(y_true)**0.5)
    return val
  return lossimage_3

def custom_loss_2():

  def lossimage_2(y_true, y_pred):

    
    #spatial_loss =  tf.reduce_mean(1-tf.image.ssim(y_pred,y_true,1))
    spatial_loss =tf.reduce_mean(tf.norm(y_pred - y_true, ord=1)) + tf.reduce_mean(1-tf.image.ssim_multiscale(y_pred,y_true,1))
    #spatial_loss =tf.reduce_mean(tf.norm(y_pred - y_true, ord='fro',axis=(1,2))) + tf.reduce_mean(1-tf.image.ssim(y_pred,y_true,1))
    
    a_b = tf.math.reduce_sum(tf.multiply(y_pred,y_true),axis=-1)
    

    mag_a = tf.sqrt(tf.reduce_sum(y_pred**2,axis=-1))
    mag_b = tf.sqrt(tf.reduce_sum(y_true**2,axis=-1))

    
    spectral_loss = tf.reduce_mean(tf.abs(a_b-tf.multiply(mag_a,mag_b))) 

    spectral_smoothness = tf.reduce_mean(tf.norm(y_pred - y_true, ord=1, axis=3))
    

    val = 3*spectral_loss + 2*spatial_loss
    return val
  return lossimage_2


class Aument_parameters(tf.keras.callbacks.Callback):
    def __init__(self, p_aum,p_step):
        super().__init__()
        self.p_aum = p_aum
        self.p_step = p_step

    def on_epoch_end(self, epoch, logs=None):
        
        if (tf.math.floormod(epoch,self.p_step)==(self.p_step-1)):
            param=self.model.layers[1].my_regularizer.parameter
            param = tf.keras.backend.get_value(param)
            self.model.layers[1].my_regularizer.parameter.assign(param * self.p_aum)
            tf.print(' regularizator ='+ str(param* self.p_aum))
    
        print(tf.keras.backend.get_value(self.model.layers[1].my_regularizer.parameter))


class save_each_epoch(tf.keras.callbacks.Callback):
    def __init__(self, checkpoint_dir):
        self.checkpoint_dir = checkpoint_dir

    
    def on_epoch_end(self, epoch, logs=None):
        print('Model Saved at: ' + self.checkpoint_dir)
        
        self.model.save_weights(self.checkpoint_dir)


from keras.callbacks import LearningRateScheduler

# This is a sample of a scheduler I used in the past
def lr_scheduler(epoch, lr):
    decay_step = 40
    if epoch % decay_step == 0 and epoch:
        lr = lr/2
        tf.print(' Learning rate ='+ str(lr))        
        return lr
    
    return lr

# Train network


In [ ]:
 
## Run Deep fusion network
# load libraries
 
 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
 
# parameters of the net

 
BATCH_SIZE = 1; EPOCHS=300; IMG_WIDTH = 512; IMG_HEIGHT = 512; L_bands = 31; dm=1/9; ds=1; 
 
# directory
#dataset_path = '/content/drive/Shareddrives/rajc990330@gmail.com/DataFusion'
dataset_path = '/content/drive/MyDrive/DeepFusion/data_FUSION'
dataset_dict = 'rad'
callback_path_fil_train = '/content/drive/MyDrive/DeepFusion/FinalUnrolledNetwork.h5'
 
#---------------------Generator-----------------------------------------------------------------------------------------

p_max = 0.5
steps = 25
p_aum = 10


# Transformations 
D = ImageDataGenerator(rotation_range=180,width_shift_range=0.2, height_shift_range=0.2,
    horizontal_flip=True)
 
params = {'dim': (IMG_WIDTH, IMG_HEIGHT, L_bands),
          'batch_size': BATCH_SIZE,
          'im_path': dataset_path, 
          'dic_img': dataset_dict,
          'shuffle': True, 
          'augment': False}  # Augmented only for training
 
HyperFiles = [
        fn  # Create full paths to images
        for fn in listdir(dataset_path)
        if isfile(join(dataset_path, fn))
           and fn.lower().endswith(('.mat'))
    ]
 
print('Training images: ',len(HyperFiles))
 
flag_tr=False
train_gen = DataGenerator(HyperFiles, flag_tr, D, **params)
 
params = {'dim': (IMG_WIDTH, IMG_HEIGHT, L_bands),
          'batch_size': BATCH_SIZE,
          'im_path': dataset_path + '/validation',
          'dic_img': dataset_dict,
          'shuffle': True,
          'augment': False}
 
HyperFiles = [
        fn  # Create full paths to images
        for fn in listdir(dataset_path + '/validation')
        if isfile(join(dataset_path + '/validation', fn))
           and fn.lower().endswith(('.mat'))
    ]
 
print('Testing images: ',len(HyperFiles))
 
flag_tr=False
val_gen = DataGenerator(HyperFiles, flag_tr, D, **params)
 
#-------------Fusion_Net_model------------------------------------------------------------------------------------------
 
'''loss_total = {'output_f1': tf.keras.losses.MeanSquaredError(), 
              'output_f2': tf.keras.losses.BinaryCrossentropy(from_logits=False)}'''
loss_total = custom_loss_2 ()
metrics_total = {'output_f1': 'MeanSquaredError'}
model = FusionNetwork_2(pretrained_weights=None, input_size=(512, 512, 31), depth=31, batch_size = BATCH_SIZE, dm=dm, ds=ds,decimation=0.5,trainable=True,parameter = 10)
 
model.load_weights(callback_path_fil_train)
 
lossWeights = {'output_f1': 0, 'output_f2': 1}
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1,
    decay_rate=0.5,
    staircase=True)


optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, amsgrad=True)
 
model.compile(optimizer=optimizer,
              loss=loss_total, metrics = [PSNR_Metric]) 
 
 
 
 
model.summary()
 
 
 
#----------------Run network--------------------------------------------------------------------------------------------
cp_callback_train = save_each_epoch(callback_path_fil_train)
callbacks = [cp_callback_train,LearningRateScheduler(lr_scheduler, verbose=1)]

#comment next line just to load the pre-tranined model

history = model.fit(x=train_gen, validation_data=val_gen, verbose=1, epochs=EPOCHS, callbacks=[callbacks])



# Test

In [ ]:
from keras.models import model_from_json


#Image =scipy.io.loadmat('/content/drive/My Drive/DeepFusion/data_FUSION/Test/BGU_162.mat')['rad'] # Test Image
Image =scipy.io.loadmat('/content/drive/Shareddrives/rajc990330@gmail.com/DataFusion/ARAD_HS_117.mat')['rad']
# Reconstruction Model

def Deep_Model_Test(pretrained_weights=None, input_size = (512,512,31), input_size_mult=(512, 512, 16),input_size_cassi=(256, 256, 31), depth=31, batch_size=4, dm=1, ds=0.5,sd=0.5,decimation=0.5,Hh=None,Hm=None,ym = None,yh = None,shots=1,trainable=True):

  # define the model input
  input_multi = Input(shape=input_size_mult)
  input_cassi = Input(shape=input_size_cassi)
  
  #Initialization
  #Xh = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(Xh)
  Xh = UpSampling2D([int(1/decimation),int(1/decimation)])(input_cassi)

  Xm = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(input_multi) 
  X0 = Add()([Xh,Xm])

  #Stage 1
  stage_1_m = Lambda(GradientMultispectral)([X0,ym,Hm,sd,dm,ds,16]) 
  stage_1_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_1_m) 

  stage_1_h = Lambda(GradientCASSI)([X0,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  stage_1_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_1_h)

  Up1 =  UpSampling2D([2,2])(X0)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)
  
  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X0,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)
  
  add_x = Subtract()([X0,conv_r5])

  PP_1 = Mu_parameter()(add_x)

  Add_G1 = Add()([stage_1_h,stage_1_m,PP_1])

  GS_1 = Lambda_parameter()(Add_G1)

  X1 = Subtract()([X0,GS_1])
  
  #Stage 2
  stage_2_m = Lambda(GradientMultispectral)([X1,ym,Hm,sd,dm,ds,16]) 
  stage_2_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_2_m) 

  stage_2_h = Lambda(GradientCASSI)([X1,yh,input_size[0],input_size[2],shots,decimation,Hh])
  #stage_2_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_2_h) 
  stage_2_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_2_h)

  Up1 =  UpSampling2D([2,2])(X1)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X1,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X1,conv_r5])
  

  PP_2 = Mu_parameter()(add_x)

  Add_G2 = Add()([stage_2_h,stage_2_m,PP_2])

  GS_2 = Lambda_parameter()(Add_G2)

  X2 = Subtract()([X1,GS_2])

  #Stage 3

  stage_3_m = Lambda(GradientMultispectral)([X2,ym,Hm,sd,dm,ds,16]) 
  stage_3_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_3_m) 

  stage_3_h = Lambda(GradientCASSI)([X2,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_3_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_3_h) 
  stage_3_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_3_h)

  Up1 =  UpSampling2D([2,2])(X2)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)
  
  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X2,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X2,conv_r5])

  PP_3 = Mu_parameter()(add_x)

  Add_G3 = Add()([stage_3_h,stage_3_m,PP_3])

  GS_3 = Lambda_parameter()(Add_G3)

  X3 = Subtract()([X2,GS_3])

  # Stage 4
  stage_4_m = Lambda(GradientMultispectral)([X3,ym,Hm,sd,dm,ds,16]) 
  stage_4_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_4_m) 

  stage_4_h = Lambda(GradientCASSI)([X3,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_4_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_4_h) 
  stage_4_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_4_h)

  Up1 =  UpSampling2D([2,2])(X3)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)
 
  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X3,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X3,conv_r5]) 

  PP_4 = Mu_parameter()(add_x)

  Add_G4 = Add()([stage_4_h,stage_4_m,PP_4])

  GS_4 = Lambda_parameter()(Add_G4)

  X4 = Subtract()([X3,GS_4])

  # Stage 5
  stage_5_m = Lambda(GradientMultispectral)([X4,ym,Hm,sd,dm,ds,16]) 
  stage_5_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_5_m) 

  stage_5_h = Lambda(GradientCASSI)([X4,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_5_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_5_h) 
  stage_5_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_5_h)

  Up1 =  UpSampling2D([2,2])(X4)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X4,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X4,conv_r5])

  PP_5 = Mu_parameter()(add_x)

  Add_G5 = Add()([stage_5_h,stage_5_m,PP_5])

  GS_5 = Lambda_parameter()(Add_G5)

  X5 = Subtract()([X4,GS_5])
  
  # Stage 6
  stage_6_m = Lambda(GradientMultispectral)([X5,ym,Hm,sd,dm,ds,16]) 
  stage_6_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_6_m) 

  stage_6_h = Lambda(GradientCASSI)([X5,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_6_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_6_h) 
  stage_6_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_6_h)

  Up1 =  UpSampling2D([2,2])(X5)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X5,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)
  
  add_x = Subtract()([X5,conv_r5]) 

  PP_6 = Mu_parameter()(add_x)

  Add_G6 = Add()([stage_6_h,stage_6_m,PP_6])

  GS_6 = Lambda_parameter()(Add_G6)

  X6 = Subtract()([X5,GS_6])
  

  # Stage 7


  stage_7_m = Lambda(GradientMultispectral)([X6,ym,Hm,sd,dm,ds,16]) 
  stage_7_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_7_m) 

  stage_7_h = Lambda(GradientCASSI)([X6,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_7_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_7_h) 
  stage_7_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_7_h)

  Up1 =  UpSampling2D([2,2])(X6)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
    
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X6,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X6,conv_r5])

  PP_7 = Mu_parameter()(add_x)

  Add_G7 = Add()([stage_7_h,stage_7_m,PP_7])

  GS_7 = Lambda_parameter()(Add_G7)

  X7 = Subtract()([X6,GS_7])
  
  #Stage 8


  stage_8_m = Lambda(GradientMultispectral)([X7,ym,Hm,sd,dm,ds,16]) 
  stage_8_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_8_m) 

  stage_8_h = Lambda(GradientCASSI)([X7,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_7_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_7_h) 
  stage_8_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_8_h)

  Up1 =  UpSampling2D([2,2])(X7)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = Add()([X7,conv_r4])

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X7,conv_r5])

  PP_8 = Mu_parameter()(add_x)

  Add_G8 = Add()([stage_8_h,stage_8_m,PP_8])

  GS_8 = Lambda_parameter()(Add_G8)

  X8 = Subtract()([X7,GS_8])

  #Stage 9

  stage_9_m = Lambda(GradientMultispectral)([X8,ym,Hm,sd,dm,ds,16]) 
  stage_9_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_9_m) 

  stage_9_h = Lambda(GradientCASSI)([X8,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_9_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_7_h) 
  stage_9_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_9_h)

  Up1 =  UpSampling2D([2,2])(X8)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)

  conv_r4 = Add()([X8,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X8,conv_r5])

  PP_9 = Mu_parameter()(add_x)

  Add_G9 = Add()([stage_9_h,stage_9_m,PP_9])

  GS_9 = Lambda_parameter()(Add_G9)

  X9 = Subtract()([X8,GS_9])
 
  #Stage 10

  stage_10_m = Lambda(GradientMultispectral)([X9,ym,Hm,sd,dm,ds,16]) 
  stage_10_m = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal',activation=None)(stage_10_m) 

  stage_10_h = Lambda(GradientCASSI)([X9,yh,input_size[0],input_size[2],shots,decimation,Hh]) 
  #stage_10_h = Conv2DTranspose(filters=depth, kernel_size=(4,4), strides=(2, 2), padding='SAME', dilation_rate=(4, 4))(stage_10_h) 
  stage_10_h = UpSampling2D([int(1/decimation),int(1/decimation)])(stage_10_h)

  Up1 =  UpSampling2D([2,2])(X9)
  
  conv_r1 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Up1)

  conv_r2 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(conv_r1)

  Down1 = MaxPool2D(pool_size=[2,2])(conv_r2)
  
  conv_r3 = Conv2D(2*depth, (3, 3), padding="same", kernel_initializer='he_normal', activation=None)(Down1)

  conv_r4 = Conv2D(depth, (3, 3), padding="same", kernel_initializer='he_normal', activation="relu")(conv_r3)

  conv_r4 = BatchNormalization()(conv_r4)
  
  conv_r4 = Add()([X9,conv_r4])

  conv_r5 = Conv2D(depth, (1, 1), padding="same", kernel_initializer='he_normal', activation=None)(conv_r4)

  add_x = Subtract()([X9,conv_r5])

  PP_10 = Mu_parameter()(add_x)

  Add_G10 = Add()([stage_10_h,stage_10_m,PP_10])

  GS_10 = Lambda_parameter()(Add_G10)

  X10 = Subtract()([X9,GS_10])
  
  model = Model([input_cassi,input_multi],X10)

  if (pretrained_weights):
    model.load_weights(pretrained_weights)
    print('loading weights generator')

  return model


# Measurements from the trained patterned 

def GetMeasurements_multi(Hm,inputs,dm,ds,sd=0.5):

    
    M = 512
    N = 512
    L = 31
    Lp = ceil(sd*L)
    inputs = tf.expand_dims(tf.reshape(tf.image.resize(tf.expand_dims(tf.reshape(inputs,[M*N,L]),-1),[M*N,Lp]),[M,N,Lp]),0)

    fr,fg,fc,fb=get_color_bases(np.linspace(400, 700, Lp)*1e-9)



    wt = Hm[0] + Hm[1] + Hm[2] + Hm[3]
    wr = tf.math.divide(Hm[0], wt)
    wg = tf.math.divide(Hm[1], wt)
    wb = tf.math.divide(Hm[2], wt)
    wc = tf.math.divide(Hm[3], wt)

    H = tf.multiply(wr, fr) + tf.multiply(wg, fg) + tf.multiply(wb, fb) + tf.multiply(wc, fc)



    if ds>dm:
      H = tf.keras.backend.repeat_elements(H, rep=int(ceil(1/dm)), axis=2)
      H = tf.keras.backend.repeat_elements(H, rep=int(ceil(1/dm)), axis=1)
      H = tf.image.resize(H,[M, N])
      

      y = tf.multiply(H,inputs)
      y = tf.expand_dims(tf.reduce_sum(y,axis=-1),axis=-1)
    elif dm<ds:
      y = tf.expand_dims(tf.reduce_sum(tf.multiply(H,inputs),axis=-1),axis=-1)
      y = tf.image.resize(y,[int(M*ds), int(N*ds)])
      y = tf.expand_dims(tf.reduce_sum(y,axis=-1),axis=-1)

    X = None
   


   
    for i in range(Lp):
   

      if X is not None:
        
        X = tf.concat([X,y],-1)        
      else:
        X = y


    X = tf.transpose(X,[0,1,2,3])

      
      
    X = tf.multiply(H, X)


    X = X / tf.math.reduce_max(X) 

    return X,y,H


# Measurements from the trained CASSI code aperture 


def GetMeasurements_cassi(Hh,inputs,shots,HO,decimation=0.5):
  L = 31
  M = int(decimation*512)

  inputs = tf.image.resize(inputs,[M,M])
  inputs = tf.expand_dims(inputs,-1)
  HO = tf.expand_dims(tf.expand_dims(tf.expand_dims(tf.expand_dims(HO,0),0),-1),-1)
  K = shots
  H = Hh
  inputs = tf.cast(inputs,dtype=tf.float32)
  # High Order CASSI Sensing Model

  inputs = tf.expand_dims(inputs,-1)
  Image2 = tf.expand_dims(tf.reduce_sum(tf.nn.conv3d(inputs,HO,strides=[1,1,1,1,1],padding='SAME'),axis=-1),0)

  Aux1 = tf.multiply(H,Image2)
  Aux1 = tf.pad(Aux1, [[0, 0], [0, 0], [0, L - 1], [0, 0],[0, 0]])

  Y = None
  for i in range(L):
      Tempo = tf.roll(Aux1, shift=i, axis=2)
      if Y is not None:
          Y = tf.concat([Y, tf.expand_dims(Tempo[:, :, :, i], -1)], axis=4)
      else:
          Y = tf.expand_dims(Tempo[:, :, :, i], -1)
  Y = tf.reduce_sum(Y, -1)
  # CASSI Transpose model (x = H'*y)
  X = None
  for i in range(L):
      Tempo = tf.roll(Y, shift=-i, axis=2)
      if X is not None:
          X = tf.concat([X, tf.expand_dims(Tempo[:, :, 0:M], -1)], axis=4)
      else:
          X = tf.expand_dims(Tempo[:, :, 0:M], -1)


  # High Order
  X = tf.transpose(X,[0,1,2,4,3])

  X2 = None
  for i in range(K):
    Aux2 = tf.expand_dims(X[:,:,:,:,i],-1)
    if X2 is not None:
      X2 =tf.concat([X2,tf.expand_dims(tf.reduce_sum(tf.nn.conv3d_transpose(Aux2,HO,[1,M,M,L,1],strides=[1,1,1,1,1],padding='SAME'),axis=-1),-1)],4)
    else:
      X2 = tf.expand_dims(tf.reduce_sum(tf.nn.conv3d_transpose(Aux2,HO,[1,M,M,L,1],strides=[1,1,1,1,1],padding='SAME'),axis=-1),-1)


  X = tf.multiply(H, X)
  X = tf.reduce_sum(X,4)

  X = X / tf.math.reduce_max(X)


  return X,Y,H

Hh = model.get_weights()[0]
Hm = model.get_weights()[1:5]


[X_m,y_m,H_m] = GetMeasurements_multi(Hm,Image,dm,ds) 
[X_c,y_c,H_c] = GetMeasurements_cassi(Hh,Image,1,[0.25,0.5,0.25])


[M_m,N_m,L_m] = tf.squeeze(X_m).shape
[M_c,N_c,L_c] = tf.squeeze(X_c).shape
modelTest = Deep_Model_Test(pretrained_weights=None, input_size_mult=(M_m,N_m,L_m),input_size_cassi=(M_c,N_c,L_c), dm=dm, ds=ds, sd=0.5,batch_size = BATCH_SIZE,decimation=0.5,Hh = H_c,Hm =H_m,ym=y_m,yh=y_c)


# Assign pretrained weights to the reconstruction model

Wrecont = model.get_weights()[5:]



modelTest.set_weights(Wrecont)
Xr = modelTest.predict([X_c,X_m])

PSNR = tf.image.psnr(Image,Xr,1)
PSNR = PSNR.numpy()[0]


# **Plot results**

In [ ]:
#Visual results
plt.gray()
plt.imshow(Image[:,:,[15,10,6]]),plt.title('Ground Truth',fontsize=20),plt.show()
plt.imshow(np.squeeze(y_m)),plt.title('Measurements Patternerd',fontsize=20),plt.show()
plt.imshow(np.squeeze(y_c[0,:,:])),plt.title('Measurements CASSI ' ,fontsize=20),plt.show()
plt.imshow(np.squeeze(xr[:,:,:,[15,10,6]])),plt.title('Reconstruction: PSNR = ' + str(PSNR) + '[dB]',fontsize=20),plt.show()

#Spectral signature reconstruction. 

plt.plot(np.squeeze(Image[400,300,:]),label='GT')
plt.plot(np.squeeze(X[-1][:,400,300,:]),label='Reconstructed')
plt.title('Spectral Signature (400,300)')
plt.legend()
plt.show()